In [1]:
!mkdir ~/.kaggle
!echo '{"username":"","key":""}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle

!kaggle datasets download -d aryashah2k/brain-tumor-segmentation-brats-2019
!unzip '/content/brain-tumor-segmentation-brats-2019.zip'

100% 2.59G/2.60G [00:39<00:00, 58.8MB/s]
100% 2.60G/2.60G [00:39<00:00, 70.8MB/s]
Archive:  /content/brain-tumor-segmentation-brats-2019.zip
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_flair.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_seg.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t1.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t1ce.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t2.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_flair.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_seg.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1.nii  
  inflating: MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1ce.nii  
 

In [1]:
!pip install tensorlayer

from preprocess import save_to_file
from metrics import dice_coefficient,dice_loss,mean_iou,sensitivity,specificity
from gen import CustomDataGen

In [2]:
from _model_swin_unet_2d import swin_unet_2d

model = swin_unet_2d((256,256,4), filter_num_begin=64, n_labels=1, depth=4, stack_num_down=2, stack_num_up=2, 
                            patch_size=(2, 2), num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=512, 
                            output_activation='Softmax', shift_window=True, name='swin_unet')

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [3]:
df = save_to_file()

train_data_frame = df[:200]
test_data_frame = df[201:]
train_gen = CustomDataGen(train_data_frame,to_fit=True)
test_gen = CustomDataGen(test_data_frame,to_fit=True)

In [4]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

BATCH_SIZE=1
EPOCHS = 30

model.compile(optimizer=Adam(learning_rate=5e-6,clipnorm=1.), loss=dice_loss, metrics=[dice_coefficient,mean_iou,sensitivity,specificity ])
callbacks = [ModelCheckpoint('_unet_brain_mri_seg.hdf5', verbose=1, save_best_only=True)]
history = model.fit(train_gen,
                          steps_per_epoch=len(train_data_frame)*15 , 
                          epochs=EPOCHS,
                          callbacks=callbacks,
                          validation_data = test_gen,
                          validation_steps=len(test_data_frame)*15
                   )

Epoch 1/30
  12/3000 [..............................] - ETA: 38:08:53 - loss: 0.9939 - dice_coefficient: 0.0061 - mean_iou: 0.2126 - sensitivity: 0.6667 - specificity: 0.0000e+00

KeyboardInterrupt: ignored